# Solar Wind Tools Run All
In this notebook we will use all the functions currently available in Solar Wind Tools to create our solar wind dataset

## Downloading
First we import the download functon.\
Then provide a file path and the start year and we end year.\
We can also provide the month we start at in the start year and the month we end at in the end year\

In this exampe we are downloading from 2000 to 2001 and including the full year for the start and end year. We then save it into a file called 'omni_1min.h5'

In [2]:
from Solar_Wind_Tools.download import download_omni_1min
download_omni_1min(fromYear=2000, toYear=2001, monthFirstYear=1, monthLastYear=12, path='./omni_1min.h5')


--2024-09-12 17:32:37--  https://cdaweb.gsfc.nasa.gov/sp_phys/data/omni/hro_1min/2000/omni_hro_1min_20000101_v01.cdf
Resolving cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)... 2001:4d0:2418:121::65, 169.154.154.65
Connecting to cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)|2001:4d0:2418:121::65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8769714 (8.4M) [application/x-cdf]
Saving to: ‘omni_hro_1min_20000101_v01.cdf’

     0K .......... .......... .......... .......... ..........  0%  259K 33s
    50K .......... .......... .......... .......... ..........  1%  504K 25s
   100K .......... .......... .......... .......... ..........  1% 1.13M 19s
   150K .......... .......... .......... .......... ..........  2%  318M 14s
   200K .......... .......... .......... .......... ..........  2%  833K 13s
   250K .......... .......... .......... .......... ..........  3% 1.13M 12s
   300K .......... .......... .......... .......... ..........  4% 72.8M 10s
   350K .......

The hdf key for the omni data is 'omni'

## Calculate Statistics
We are now going to calculate statistics using a 10 minute window.\
We provide the file path of the omni data. We set the save path as the same so that it uses the same file.\
The load key is the hdf file key for the omni data.
The key is the key the new dataset is saved to that has the statistics. We have set it to 'omni_window'

In [3]:
from Solar_Wind_Tools.window import statistics
statistics(file_path='./omni_1min.h5', save_path='./omni_1min.h5', window=10, load_key='omni', key='omni_window')

  0% (0 of 4) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 25% (1 of 4) |######                    | Elapsed Time: 0:00:02 ETA:   0:00:06
 50% (2 of 4) |#############             | Elapsed Time: 0:00:02 ETA:   0:00:02
 75% (3 of 4) |###################       | Elapsed Time: 0:00:03 ETA:   0:00:01
100% (4 of 4) |##########################| Elapsed Time: 0:00:03 Time:  0:00:03


In the 'omni_1min.h5' file we now have two keys omni with the original data and omni_window which has the original data and the statistics
## Coupling Functions
So far only the newell coupling function has been implemented. We will calculate its value and calculate the mean across the 10 minute window.
If a window is not provided then the mean is not calculated

In [4]:
from Solar_Wind_Tools.window import coupling
coupling('./omni_1min.h5', './omni_1min.h5', window=10, load_key='omni_window', key='omni_window')


## Dipole Tilt
We use the package at available at: https://github.com/klaundal/dipole' to calculate the dipole tilt.\

We use the same load and save file and alter the same hdf key.


In [5]:
from Solar_Wind_Tools.window import dipole
dipole('./omni_1min.h5', './omni_1min.h5', load_key='omni_window', key='omni_window')

/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/v

## Time Shifting
This useful if you want to shift the time to create a delay or lag between the solar wind data and the data you are trying to associate it with.
time shift is in minutes we have you used 10 minutes. Positive value will make a time step be associated with a value from 10 minutes ago.\
This shift is only applied to statistics columns.

In [6]:
from Solar_Wind_Tools.window import time_shift
time_shift('./omni_1min.h5', './omni_1min.h5', shift=10, load_key='omni_window', key='omni_window')


## Shortcut to running multiple
We have provided a run all function to do a one line running of all functions.\
You provide a list of strings for all the functions you want to run. The options are: ['Download', 'Statistics', 'Coupling', 'Dipole', 'Time Shift'].\

Here we download from 2000 to 2001. We calculate the statistics using a 2 hour window. We then shift the statistics by 20 minutes.

In [7]:
from Solar_Wind_Tools.run_all import run_all
run_all('./test_omni.hdf5',
        key='omni_window_2h_20', window=2*60, shift=20, start_year= 2000, 
        end_year=2001, run=['Download', 'Statistics', 'Coupling', 'Dipole', 'Time Shift'])

--2024-09-12 17:36:38--  https://cdaweb.gsfc.nasa.gov/sp_phys/data/omni/hro_1min/2000/omni_hro_1min_20000101_v01.cdf
Resolving cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)... 2001:4d0:2418:121::65, 169.154.154.65
Connecting to cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)|2001:4d0:2418:121::65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8769714 (8.4M) [application/x-cdf]
Saving to: ‘omni_hro_1min_20000101_v01.cdf’

     0K .......... .......... .......... .......... ..........  0%  259K 33s
    50K .......... .......... .......... .......... ..........  1%  482K 25s
   100K .......... .......... .......... .......... ..........  1%  502K 22s
   150K .......... .......... .......... .......... ..........  2%  587M 17s
   200K .......... .......... .......... .......... ..........  2%  479K 17s
   250K .......... .......... .......... .......... ..........  3%  361M 14s
   300K .......... .......... .......... .......... ..........  4%  499M 12s
   350K .......

Statistics


  0% (0 of 4) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 25% (1 of 4) |######                    | Elapsed Time: 0:00:01 ETA:   0:00:05
 50% (2 of 4) |#############             | Elapsed Time: 0:00:03 ETA:   0:00:03
 75% (3 of 4) |###################       | Elapsed Time: 0:00:05 ETA:   0:00:01
100% (4 of 4) |##########################| Elapsed Time: 0:00:07 Time:  0:00:07



 Time elapsed: 0Days:0Hrs:0Mins:36Secs 

Coupling

 Time elapsed: 0Days:0Hrs:0Mins:28Secs 

Dipole


/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/vscode/lib/python3.10/site-packages/ppigrf/ppigrf.py:139: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  delta_year = pd.TimedeltaIndex((fracyear - year)*(365 + is_leapyear(year)), unit = 'D')
/home/simon/anaconda3/envs/v


 Time elapsed: 0Days:0Hrs:0Mins:29Secs 

Time Shift

 Time elapsed: 0Days:0Hrs:0Mins:40Secs 



In [8]:
import pandas as pd
pd.read_hdf('./test_omni.hdf5', key='omni')

,Epoch,YR,Day,HR,Minute,IMF,PLS,IMF_PTS,PLS_PTS,percent_interp,...,BSN_y,BSN_z,AE_INDEX,AL_INDEX,AU_INDEX,SYM_D,SYM_H,ASY_D,ASY_H,PC_N_INDEX
2000-01-01 00:00:00,6.311390e+13,2000,1,0,0,71.0,NaN,5.0,NaN,100.0,...,-0.69,0.84,668.0,-486.0,182.0,-6,-44,61,48,6.22
2000-01-01 00:01:00,6.311390e+13,2000,1,0,1,71.0,71.0,26.0,6.0,100.0,...,-0.69,0.81,638.0,-487.0,151.0,-6,-45,64,48,6.47
2000-01-01 00:02:00,6.311390e+13,2000,1,0,2,71.0,71.0,15.0,9.0,100.0,...,-0.48,1.01,666.0,-527.0,139.0,-7,-45,67,48,6.60
2000-01-01 00:03:00,6.311390e+13,2000,1,0,3,71.0,71.0,1.0,3.0,100.0,...,-0.36,1.07,615.0,-474.0,141.0,-7,-45,69,48,6.54
2000-01-01 00:04:00,6.311390e+13,2000,1,0,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,554.0,-418.0,136.0,-8,-45,69,47,6.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-12-31 23:55:00,6.317706e+13,2001,365,23,55,71.0,71.0,9.0,4.0,100.0,...,-2.20,0.71,109.0,-30.0,79.0,1,-32,14,22,1.10
2001-12-31 23:56:00,6.317706e+13,2001,365,23,56,71.0,71.0,2.0,3.0,100.0,...,-1.80,0.73,107.0,-26.0,81.0,1,-32,14,22,1.17
2001-12-31 23:57:00,6.317706e+13,2001,365,23,57,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,109.0,-22.0,87.0,1,-32,14,22,1.16
2001-12-31 23:58:00,6.317706e+13,2001,365,23,58,71.0,NaN,1.0,NaN,100.0,...,-1.82,0.68,102.0,-14.0,88.0,1,-32,14,22,1.19


## We use HDF files because they are searchable without loading it all into ram

In [9]:
pd.read_hdf('./test_omni.hdf5', key='omni_window_2h_20', where="index>='2001-01-01T00:00'")

,Epoch,YR,Day,HR,Minute,IMF,PLS,IMF_PTS,PLS_PTS,percent_interp,...,BZ_GSM_SEM,points,Clock_GSM,Circular_Variance_GSM,Clock_GSM_SEM,Clock_GSM_Mean,Clock_GSM_Median,Newell_Epsilon,Newell_Epsilon_Mean,Dipole_Tilt
2001-01-01 00:00:00,6.314553e+13,2001,1,0,0,51.0,NaN,3.0,NaN,100.0,...,0.087981,97.0,2.308611,0.142520,2.196217,1.604486,1.459768,NaN,1.250785,-25.761152
2001-01-01 00:01:00,6.314553e+13,2001,1,0,1,51.0,NaN,3.0,NaN,67.0,...,0.083804,97.0,2.411415,0.134064,2.130061,1.587784,1.459768,NaN,1.193398,-25.805106
2001-01-01 00:02:00,6.314553e+13,2001,1,0,2,51.0,51.0,7.0,3.0,100.0,...,0.081548,97.0,1.376837,0.129057,2.089906,1.575442,1.462380,0.373619,1.146815,-25.849015
2001-01-01 00:03:00,6.314553e+13,2001,1,0,3,51.0,51.0,16.0,3.0,100.0,...,0.080558,97.0,0.485035,0.127183,2.074683,1.567136,1.456316,0.036131,1.137234,-25.892877
2001-01-01 00:04:00,6.314553e+13,2001,1,0,4,51.0,51.0,10.0,3.0,100.0,...,0.079737,97.0,0.708626,0.126109,2.065904,1.558643,1.449444,0.081266,1.125429,-25.936693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-12-31 23:55:00,6.317706e+13,2001,365,23,55,71.0,71.0,9.0,4.0,100.0,...,0.139280,116.0,-2.363986,0.025022,0.841499,-2.046228,-2.028968,5.497880,8.920994,-25.564910
2001-12-31 23:56:00,6.317706e+13,2001,365,23,56,71.0,71.0,2.0,3.0,100.0,...,0.135052,116.0,-2.330819,0.023205,0.810373,-2.040842,-2.030326,7.142823,8.845682,-25.609074
2001-12-31 23:57:00,6.317706e+13,2001,365,23,57,NaN,NaN,NaN,NaN,NaN,...,0.131840,116.0,NaN,0.022295,0.794322,-2.037474,-2.028959,NaN,8.792637,-25.653197
2001-12-31 23:58:00,6.317706e+13,2001,365,23,58,71.0,NaN,1.0,NaN,100.0,...,0.129781,116.0,-2.176490,0.021817,0.785760,-2.034737,-2.027583,NaN,8.738808,-25.697279
